<a href="https://colab.research.google.com/github/CuriousCaliBoi/AgniKai/blob/main/CS25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crawl:  
 • 01_MLP.ipynb – 2-layer MLP on MNIST/CIFAR  
 • 02_Attn.ipynb – single-head attention + heatmap  

Walk:  
 • 03_MiniTF.ipynb – 2–3 Transformer blocks on tiny LM  
 • 04_ProbeOSS.ipynb – load “gpt2”/“bert-base” & inspect Wq/Wk/Wv  

Jog:  
 • 05_FT_Classification.ipynb – fine-tune BERT/GPT for sentiment  
 • 06_AttnViz.ipynb – plot real head attention on example text  

Sprint:  
 • 07_EfficientTF.ipynb – swap in Performer/Longformer & benchmark  
 • 08_LongContext.ipynb – load sparse/linear OSS models (e.g. Bloom, BigBird) on 4K+ tokens  

⏱️ Total: ~2 weeks part-time (9–11 days full-days)  
Drive mount + git clone → paste above into new Colab README and go!```

In [ ]:
# 01_MLP.ipynb

"""
## Two-layer MLP on MNIST and CIFAR-10
This notebook builds and trains a simple 2-layer multilayer perceptron (MLP) on MNIST and CIFAR-10.
Usage: copy and paste into Colab, then run cells sequentially.
"""

# Cell 1: Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Cell 2: Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Cell 3: Hyperparameters
batch_size = 128
learning_rate = 1e-3
num_epochs = 10
hidden_dim = 512

# Cell 4: Data loaders
# MNIST transforms
mnist_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# CIFAR-10 transforms (flatten color channels)
cifar_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

# Choose dataset: uncomment one
# dataset_name = 'MNIST'
# train_dataset = datasets.MNIST(root='./data', train=True, transform=mnist_transform, download=True)
# test_dataset  = datasets.MNIST(root='./data', train=False, transform=mnist_transform, download=True)

dataset_name = 'CIFAR10'
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=cifar_transform, download=True)
test_dataset  = datasets.CIFAR10(root='./data', train=False, transform=cifar_transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

# Determine input and output dimensions
if dataset_name == 'MNIST':
    input_dim = 28*28
    num_classes = 10
else:
    input_dim = 32*32*3
    num_classes = 10

# Cell 5: Model definition
class TwoLayerMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(TwoLayerMLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = TwoLayerMLP(input_dim, hidden_dim, num_classes).to(device)
print(model)

# Cell 6: Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Cell 7: Training loop
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Stats
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    epoch_loss = running_loss / total
    epoch_acc  = correct / total
    return epoch_loss, epoch_acc

# Cell 8: Evaluation loop
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    epoch_loss = running_loss / total
    epoch_acc  = correct / total
    return epoch_loss, epoch_acc

# Cell 9: Run training and evaluation
best_acc = 0.0
for epoch in range(1, num_epochs+1):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc   = evaluate(model, test_loader, criterion, device)
    print(f"Epoch [{epoch}/{num_epochs}]"
          f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}"
          f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    # Save best
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), f"best_mlp_{dataset_name}.pth")

print(f"Best test accuracy: {best_acc:.4f}")